<a href="https://colab.research.google.com/github/dolmani38/non-linear-regression/blob/master/Regession_test_0804.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lime

import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers import Input, Embedding, Dense
from keras.models import Model
from keras.callbacks import Callback
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from lime import lime_tabular, lime_text
from keras.utils import to_categorical
import datetime
import random

# Bayesian Methods for Hackers style sheet
plt.style.use('bmh')

np.random.seed(1234567890)

from keras import backend as K

def r2(y_true, y_pred):
    """
    # custom R2-score metrics for keras backend
    :param y_true: 실측 데이터
    :param y_pred: 모델에 의한 예측 데이테
    :return: R^2 value, 이 값이 높다고 (예:0.99)해서 예측 값이 정확 하다고 할 수 는 없음...
    """
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res / (SS_tot + K.epsilon()))

def soft_100_acc(y_true, y_pred):
    delta = 100
    return K.mean(K.less_equal(K.abs(y_true-y_pred),delta))

def soft_300_acc(y_true, y_pred):
    delta = 300
    return K.mean(K.less_equal(K.abs(y_true-y_pred),delta))

def structure(df):
    """
    DataFrame의 column 등 전반적인 구조를 표출 한다.
    :param df:
    :return:
    """
    df1 = df.describe(include='all').T
    df1['type'] = df.dtypes
    df1['null count'] = df.isnull().sum()
    if 'freq' in df1.columns:
        df2 = df1[
            ['type', 'count', 'null count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'unique', 'top',
             'freq']]
    else:
        df2 = df1[['type', 'count', 'null count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]

    return df2

class VerboseCallback(keras.callbacks.Callback):
    """
    kerase.model.fit 함수에서, log 출력을 위한 callback
    """
    def __init__(self, epoch):
        self.epoch = epoch
        self.progress("learning:", 0, [])

    def progress(self, text, current=0, log=None):
        g = self.epoch - current
        f = 50 / self.epoch
        c = int(current * f)
        pg = ['=' for i in range(c)] + ['.' for i in range(50 - c)]
        print("\r" + str(text) + "{}/{}[{}]{}".format(current, self.epoch, ''.join(pg), ('' if log is None else str(log).strip())),
              end="", flush=True)
        if self.epoch - current == 0:
            print("")

    def on_epoch_end(self, epoch, logs=None):
        self.progress("learning:", epoch + 1, logs)


class PeriodicLogger(Callback):
    """
    A helper callback class that only prints the losses once in 'display' epochs
    """
    def __init__(self, display=100):
        self.display = display

    def on_train_begin(self, logs={}):      
        self.epochs = 0    

    def on_epoch_end(self, batch, logs={}):    
        self.epochs += 1     
        if self.epochs % self.display == 0:
            print ("Epoch: %d - %s" % (self.epochs, str(logs)))
 
periodic_logger_50 = PeriodicLogger(50)
periodic_logger_250 = PeriodicLogger(250)
periodic_logger_1000 = PeriodicLogger(1000)

In [2]:
def nafyc_func(row):
    code,real_yy,rate = row['CODE'],row['REAL_YY'],row['COM_RATE']
    return real_yy * rate

In [6]:
def create_samples(n_samples):
 
    samples = []
    for i in range(n_samples):
      code = np.random.choice(range(1000,1520))
      real_yy = random.choice(range(1000)) * 1000
      rate = np.random.choice([0.20,0.25,0.30,0.33,0.65,0.70,0.75,0.80,0.85])
      samples.append([code,real_yy,rate])
        
    return pd.DataFrame(samples, columns=['CODE','REAL_YY','COM_RATE'])

In [8]:
dataset = create_samples(n_samples=10000)
dataset['NAFYC'] = dataset.apply(nafyc_func, axis=1)
dataset

,CODE,REAL_YY,COM_RATE,NAFYC
0,1374,994000,0.33,328020.0
1,1512,252000,0.33,83160.0
2,1412,527000,0.70,368900.0
3,1126,328000,0.33,108240.0
4,1503,526000,0.70,368200.0
...,...,...,...,...
9995,1142,874000,0.33,288420.0
9996,1388,230000,0.33,75900.0
9997,1295,325000,0.75,243750.0
9998,1299,508000,0.85,431800.0


In [9]:
structure(dataset)

,type,count,null count,mean,std,min,25%,50%,75%,max
CODE,int64,10000.0,0,1256.735200,150.361402,1000.0,1126.0,1256.00,1387.25,1519.00
REAL_YY,int64,10000.0,0,497909.900000,289524.166878,0.0,247000.0,495000.00,748000.00,999000.00
COM_RATE,float64,10000.0,0,0.537184,0.246746,0.2,0.3,0.65,0.75,0.85
NAFYC,float64,10000.0,0,268397.467000,212103.289566,0.0,100500.0,204800.00,410250.00,849150.00


In [12]:
dataset = dataset.astype({"CODE": object})

In [13]:
structure(dataset)

,type,count,null count,mean,std,min,25%,50%,75%,max,unique,top,freq
CODE,object,10000.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,520.0,1171.0,35.0
REAL_YY,int64,10000.0,0,497909.900000,289524.166878,0.0,247000.0,495000.00,748000.00,999000.00,NaN,NaN,NaN
COM_RATE,float64,10000.0,0,0.537184,0.246746,0.2,0.3,0.65,0.75,0.85,NaN,NaN,NaN
NAFYC,float64,10000.0,0,268397.467000,212103.289566,0.0,100500.0,204800.00,410250.00,849150.00,NaN,NaN,NaN


In [14]:
# zscore norm

ds = dataset[['REAL_YY','COM_RATE']]

_mean = ds.mean(axis=0)
_std = ds.std(axis=0)

ds = ds - _mean
ds /= _std

dataset.update(ds)
dataset

,CODE,REAL_YY,COM_RATE,NAFYC
0,1374,1.713467,-0.839665,328020.0
1,1512,-0.849359,-0.839665,83160.0
2,1412,0.100476,0.659852,368900.0
3,1126,-0.586859,-0.839665,108240.0
4,1503,0.097022,0.659852,368200.0
...,...,...,...,...
9995,1142,1.298994,-0.839665,288420.0
9996,1388,-0.925346,-0.839665,75900.0
9997,1295,-0.597221,0.862490,243750.0
9998,1299,0.034851,1.267765,431800.0


In [15]:
class EmbeddingMapping():
    """
    Helper class for handling categorical variables
    An instance of this class should be defined for each categorical variable we want to use.
    """
    def __init__(self, series):
        # get a list of unique values
        values = series.unique().tolist()
        
        # Set a dictionary mapping from values to integer value
        # In our example this will be {'Mercaz': 1, 'Old North': 2, 'Florentine': 3}
        self.embedding_dict = {value: int_value+1 for int_value, value in enumerate(values)}
        
        # The num_values will be used as the input_dim when defining the embedding layer. 
        # It will also be returned for unseen values 
        self.num_values = len(values) + 1

    def get_mapping(self, value):
        # If the value was seen in the training set, return its integer mapping
        if value in self.embedding_dict:
            return self.embedding_dict[value]
        
        # Else, return the same integer for unseen values
        else:
            return self.num_values

In [16]:
code_mapping = EmbeddingMapping(dataset['CODE'])
dataset = dataset.assign(code_mapping=dataset['CODE'].apply(code_mapping.get_mapping))
dataset

,CODE,REAL_YY,COM_RATE,NAFYC,code_mapping
0,1374,1.713467,-0.839665,328020.0,1
1,1512,-0.849359,-0.839665,83160.0,2
2,1412,0.100476,0.659852,368900.0,3
3,1126,-0.586859,-0.839665,108240.0,4
4,1503,0.097022,0.659852,368200.0,5
...,...,...,...,...,...
9995,1142,1.298994,-0.839665,288420.0,225
9996,1388,-0.925346,-0.839665,75900.0,334
9997,1295,-0.597221,0.862490,243750.0,42
9998,1299,0.034851,1.267765,431800.0,258


In [17]:
# trainset과 validationset 나누고... 기타 정리

X_train_org, X_val_org, Y_train, Y_val = train_test_split(dataset[['REAL_YY','code_mapping']],
    dataset['NAFYC'], test_size=0.1, random_state=0)

# 학습 대상 변수만 선택
X_train_continuous = X_train_org[['REAL_YY']]
X_train_categorical = X_train_org[['code_mapping']]

X_val_continuous  = X_val_org[['REAL_YY']]
X_val_categorical = X_val_org[['code_mapping']]


In [18]:
# Define the embedding input
code_input = Input(shape=(1,), dtype='int32') 
output_dim=X_train_categorical['code_mapping'].max()*8
code_embedings = Embedding(output_dim=output_dim, input_dim=X_train_categorical['code_mapping'].max()+1)(code_input)
code_embedings = keras.layers.Reshape((output_dim,))(code_embedings)

In [19]:
# Define the continuous variables input (just like before)
continuous_input = Input(shape=(X_train_continuous.shape[1], ))

# Concatenate continuous and embeddings inputs
all_input = keras.layers.concatenate([continuous_input, code_embedings])
print(all_input)

Tensor("concatenate/concat:0", shape=(None, 4161), dtype=float32)


In [21]:
# Define the model
dense1 = Dense(all_input.shape[1]*3, activation='relu')(all_input)
dense2 = Dense(5, activation='relu')(dense1)
predictions = Dense(1)(dense2)

# Note using the input object 'area_input' not 'area_embeddings'
model = Model(inputs=[continuous_input, code_input], outputs=predictions)
model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=.8, beta_1=0.9, beta_2=0.999, decay=1e-03, amsgrad=True),metrics=[soft_100_acc,soft_300_acc,r2])
model.summary()

Model: "functional_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 4160)      2167360     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 4160)         0           embedding[0][0]                  
_______________________________________________________________________________________

In [ ]:
epochs = 1000

# Note continuous and categorical columns are inserted in the same order as defined in all_inputs
history = model.fit([X_train_continuous, X_train_categorical['code_mapping']], Y_train, 
          epochs=epochs, batch_size=128, 
          callbacks=[periodic_logger_50], verbose=0,
          validation_data=([X_val_continuous, X_val_categorical['code_mapping']], Y_val))